In [1]:
import os
import pickle

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from src.data_preparation.data_preprocessing import DataReshaperLSTM
from src.visualization.charts import ModelCharts
from src.visualization.metrics import PredictionEvaluator
from src.config import Config

## Load the data

In [2]:
config = Config()
variant_co2 = 'co2_per_capita'

#lstm
train_lstm_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/train.csv')).set_index(["year", config.additional_index]).drop(columns=['country_order'])
test_lstm_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/test.csv')).set_index(["year", config.additional_index]).drop(columns=['country_order'])

with open(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/data_preprocessor_lstm.pkl'), 'rb') as f:
    data_preprocessor_lstm = pickle.load(f)

#residual arimax
train_arimax_residuals_df = pd.read_csv(os.path.join(config.output_cleaned_hybrid, f'arimax_lstm/{variant_co2}/residuals_train.csv')).set_index(["year", "country"]).rename_axis(index={"country": config.additional_index})
test_arimax_residuals_df = pd.read_csv(os.path.join(config.output_cleaned_hybrid, f'arimax_lstm/{variant_co2}/residuals_test.csv')).set_index(["year", "country"]).rename_axis(index={"country": config.additional_index})
train_arimax_residuals_preprocessed, test_arimax_residuals_preprocessed = data_preprocessor_lstm.preprocess_data(train_arimax_residuals_df, test_arimax_residuals_df)

In [3]:
train_lstm_df

,,country_t-2,population_t-2,gdp_t-2,temperature_change_from_co2_t-2,total_ghg_t-2,cement_co2_t-2,coal_co2_t-2,flaring_co2_t-2,gas_co2_t-2,land_use_change_co2_t-2,...,gdp_t-1,temperature_change_from_co2_t-1,total_ghg_t-1,cement_co2_t-1,coal_co2_t-1,flaring_co2_t-1,gas_co2_t-1,land_use_change_co2_t-1,oil_co2_t-1,co2
year,country_index,,,,,,,,,,,,,,,,,,,,,
1853,Stany Zjednoczone,34,0.016985,0.078879,0.000000,0.809861,0.000000,0.065960,0.000000,0.000000,0.489399,...,0.083750,0.054411,0.826034,0.000000,0.069644,0.000000,0.000000,0.501986,0.000000,0.495427
1854,Stany Zjednoczone,34,0.017544,0.083750,0.054411,0.826034,0.000000,0.069644,0.000000,0.000000,0.501986,...,0.090790,0.052824,0.807592,0.000000,0.076120,0.000000,0.000000,0.494294,0.000000,0.500984
1855,Stany Zjednoczone,34,0.018120,0.090790,0.052824,0.807592,0.000000,0.076120,0.000000,0.000000,0.494294,...,0.090836,0.102565,0.813081,0.000000,0.081242,0.000000,0.000000,0.498800,0.000000,0.495014
1856,Stany Zjednoczone,34,0.018712,0.090836,0.102565,0.813081,0.000000,0.081242,0.000000,0.000000,0.498800,...,0.088710,0.099573,0.799701,0.000000,0.090770,0.000000,0.000000,0.491227,0.000000,0.495772
1857,Stany Zjednoczone,34,0.019322,0.088710,0.099573,0.799701,0.000000,0.090770,0.000000,0.000000,0.491227,...,0.090730,0.096669,0.799440,0.000000,0.092454,0.000000,0.000000,0.491663,0.000000,0.488626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,Szwecja,35,0.005112,0.633681,0.310355,0.169580,0.160121,0.083308,0.001888,0.034710,0.056927,...,0.657400,0.308734,0.176222,0.176169,0.082777,0.002145,0.035107,0.062237,0.286368,0.123911
1997,Szwecja,35,0.005147,0.657400,0.308734,0.176222,0.186143,0.082777,0.002145,0.035107,0.062237,...,0.668932,0.308244,0.184783,0.166254,0.091217,0.002047,0.037824,0.059806,0.302837,0.118362
1998,Szwecja,35,0.005158,0.668932,0.308244,0.184783,0.175666,0.091217,0.002047,0.037824,0.059806,...,0.688825,0.308063,0.176055,0.149879,0.079360,0.002274,0.038312,0.061110,0.280898,0.119892


In [4]:
train_arimax_residuals_preprocessed

,,residual
year,country_index,
1853,Stany Zjednoczone,0.783548
1854,Stany Zjednoczone,0.399894
1855,Stany Zjednoczone,0.397765
1856,Stany Zjednoczone,0.401150
1857,Stany Zjednoczone,0.398933
...,...,...
1996,Szwecja,0.403939
1997,Szwecja,0.400457
1998,Szwecja,0.402859


In [5]:
#combine train df
train_hybrid_df = pd.merge(train_arimax_residuals_preprocessed, train_lstm_df, left_index=True, right_index=True)
train_hybrid_df

,,residual,country_t-2,population_t-2,gdp_t-2,temperature_change_from_co2_t-2,total_ghg_t-2,cement_co2_t-2,coal_co2_t-2,flaring_co2_t-2,gas_co2_t-2,...,gdp_t-1,temperature_change_from_co2_t-1,total_ghg_t-1,cement_co2_t-1,coal_co2_t-1,flaring_co2_t-1,gas_co2_t-1,land_use_change_co2_t-1,oil_co2_t-1,co2
year,country_index,,,,,,,,,,,,,,,,,,,,,
1853,Stany Zjednoczone,0.783548,34,0.016985,0.078879,0.000000,0.809861,0.000000,0.065960,0.000000,0.000000,...,0.083750,0.054411,0.826034,0.000000,0.069644,0.000000,0.000000,0.501986,0.000000,0.495427
1854,Stany Zjednoczone,0.399894,34,0.017544,0.083750,0.054411,0.826034,0.000000,0.069644,0.000000,0.000000,...,0.090790,0.052824,0.807592,0.000000,0.076120,0.000000,0.000000,0.494294,0.000000,0.500984
1855,Stany Zjednoczone,0.397765,34,0.018120,0.090790,0.052824,0.807592,0.000000,0.076120,0.000000,0.000000,...,0.090836,0.102565,0.813081,0.000000,0.081242,0.000000,0.000000,0.498800,0.000000,0.495014
1856,Stany Zjednoczone,0.401150,34,0.018712,0.090836,0.102565,0.813081,0.000000,0.081242,0.000000,0.000000,...,0.088710,0.099573,0.799701,0.000000,0.090770,0.000000,0.000000,0.491227,0.000000,0.495772
1857,Stany Zjednoczone,0.398933,34,0.019322,0.088710,0.099573,0.799701,0.000000,0.090770,0.000000,0.000000,...,0.090730,0.096669,0.799440,0.000000,0.092454,0.000000,0.000000,0.491663,0.000000,0.488626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,Szwecja,0.403939,35,0.005112,0.633681,0.310355,0.169580,0.160121,0.083308,0.001888,0.034710,...,0.657400,0.308734,0.176222,0.176169,0.082777,0.002145,0.035107,0.062237,0.286368,0.123911
1997,Szwecja,0.400457,35,0.005147,0.657400,0.308734,0.176222,0.186143,0.082777,0.002145,0.035107,...,0.668932,0.308244,0.184783,0.166254,0.091217,0.002047,0.037824,0.059806,0.302837,0.118362
1998,Szwecja,0.402859,35,0.005158,0.668932,0.308244,0.184783,0.175666,0.091217,0.002047,0.037824,...,0.688825,0.308063,0.176055,0.149879,0.079360,0.002274,0.038312,0.061110,0.280898,0.119892


In [6]:
#combine test df
test_hybrid_df = pd.merge(test_arimax_residuals_preprocessed, test_lstm_df, left_index=True, right_index=True)
test_hybrid_df

,,residual,country_t-2,population_t-2,gdp_t-2,temperature_change_from_co2_t-2,total_ghg_t-2,cement_co2_t-2,coal_co2_t-2,flaring_co2_t-2,gas_co2_t-2,...,gdp_t-1,temperature_change_from_co2_t-1,total_ghg_t-1,cement_co2_t-1,coal_co2_t-1,flaring_co2_t-1,gas_co2_t-1,land_use_change_co2_t-1,oil_co2_t-1,co2
year,country_index,,,,,,,,,,,,,,,,,,,,,
1989,Stany Zjednoczone,0.402833,34,0.186605,0.791559,0.846847,0.508303,0.169824,0.452087,0.001514,0.563915,...,0.817759,0.850208,0.524219,0.160594,0.469840,0.001729,0.583915,0.082995,0.517934,0.308212
1990,Stany Zjednoczone,0.401508,34,0.188312,0.817759,0.850208,0.524219,0.169686,0.469840,0.001729,0.583915,...,0.838605,0.853340,0.526559,0.159209,0.470866,0.001702,0.616149,0.082725,0.512904,0.305718
1991,Stany Zjednoczone,0.396525,34,0.190056,0.838605,0.853340,0.526559,0.168223,0.470866,0.001702,0.616149,...,0.844259,0.860384,0.518755,0.158555,0.466174,0.010276,0.607288,0.083405,0.493612,0.297345
1992,Stany Zjednoczone,0.408788,34,0.192119,0.844259,0.860384,0.518755,0.167532,0.466174,0.010276,0.607288,...,0.831598,0.860616,0.506536,0.153140,0.456131,0.009935,0.611097,0.080776,0.474745,0.297198
1993,Stany Zjednoczone,0.411024,34,0.194488,0.831598,0.860616,0.506536,0.161811,0.456131,0.009935,0.611097,...,0.850993,0.860721,0.509469,0.152479,0.455670,0.009743,0.625705,0.078382,0.480823,0.298140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,Szwecja,0.402708,35,0.006089,1.034912,0.406395,0.152356,0.187046,0.053815,0.004475,0.027086,...,1.042982,0.401701,0.146722,0.189482,0.051773,0.004556,0.027867,0.075373,0.164163,0.101755
2020,Szwecja,0.400023,35,0.006179,1.042982,0.401701,0.146722,0.200210,0.051773,0.004556,0.027867,...,1.053059,0.397648,0.142694,0.158390,0.051390,0.002930,0.026035,0.075950,0.159296,0.096965
2021,Szwecja,0.405910,35,0.006258,1.053059,0.397648,0.142694,0.167357,0.051390,0.002930,0.026035,...,1.022503,0.394784,0.133416,0.147399,0.041803,0.002415,0.020940,0.076321,0.146172,0.099935


## Reshape the data

In [7]:
data_resherper = DataReshaperLSTM()
x_train, x_test, y_train, y_test = data_resherper.reshape_data(train_hybrid_df, test_hybrid_df)

## Build the model

In [8]:
input_shape = (x_train.shape[1], x_train.shape[2])
output_units = 1

model = Sequential([
    LSTM(50, input_shape=input_shape, return_sequences = True),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(output_units, activation='relu')
])

model.compile(optimizer="adam", loss="mean_squared_error")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             14800     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 35,051
Trainable params: 35,051
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [9]:
history = model.fit(x_train, y_train, epochs=config.epochs, batch_size=config.batch_size)

Epoch 1/15
3153/3153 [==============================] - 4s 824us/step - loss: 0.0044
Epoch 2/15
3153/3153 [==============================] - 2s 679us/step - loss: 0.0024
Epoch 3/15
3153/3153 [==============================] - 2s 707us/step - loss: 0.0021
Epoch 4/15
3153/3153 [==============================] - 2s 673us/step - loss: 0.0019
Epoch 5/15
3153/3153 [==============================] - 2s 793us/step - loss: 0.0017
Epoch 6/15
3153/3153 [==============================] - 2s 699us/step - loss: 0.0017
Epoch 7/15
3153/3153 [==============================] - 2s 669us/step - loss: 0.0016
Epoch 8/15
3153/3153 [==============================] - 2s 694us/step - loss: 0.0017
Epoch 9/15
3153/3153 [==============================] - 2s 712us/step - loss: 0.0016
Epoch 10/15
3153/3153 [==============================] - 2s 719us/step - loss: 0.0014
Epoch 11/15
3153/3153 [==============================] - 2s 698us/step - loss: 0.0016
Epoch 12/15
3153/3153 [==============================] - 2s 698

In [10]:
# Save the model
model.save(os.path.join(config.models_folder, f'{variant_co2}_arimax_lstm_model.h5'))

# Evaluate the model on the test set
loss = model.evaluate(x_test, y_test, verbose = 0)

## Predictions

In [11]:
train_predictions = model.predict(x_train)
test_predictions = model.predict(x_test)

inverted_data_predicted_train_y = data_preprocessor_lstm.inverse_transform_data(train_predictions, train_predictions.shape[0], train_hybrid_df.shape[1]-2)
inverted_data_train_y = data_preprocessor_lstm.inverse_transform_data(y_train, train_predictions.shape[0], train_hybrid_df.shape[1]-2)

inverted_data_predicted_test_y = data_preprocessor_lstm.inverse_transform_data(test_predictions, test_predictions.shape[0], test_hybrid_df.shape[1]-2)
inverted_data_test_y = data_preprocessor_lstm.inverse_transform_data(y_test, test_predictions.shape[0], test_hybrid_df.shape[1]-2)

26/26 [==============================] - 0s 410us/step


In [12]:
train_df_reset = train_hybrid_df.reset_index()
test_df_reset = test_hybrid_df.reset_index()

co2_predicted_train = inverted_data_predicted_train_y[:, -1]
co2_actual_train = inverted_data_train_y[:, -1]
co2_predicted_test = inverted_data_predicted_test_y[:, -1]
co2_actual_test = inverted_data_test_y[:, -1]

# Create DataFrames for train and test
train_results = pd.DataFrame({
    "country": train_df_reset["country_index"].values,
    "year": train_df_reset["year"].values,
    "co2_predicted": co2_predicted_train,
    "co2_actual": co2_actual_train
})

test_results = pd.DataFrame({
    "country": test_df_reset["country_index"].values,
    "year": test_df_reset["year"].values,
    "co2_predicted": co2_predicted_test,
    "co2_actual": co2_actual_test
})


train_results.to_csv(os.path.join(config.predictions_hybrid_arimax_lstm, f'arimax_lstm_{variant_co2}_train.csv'))
test_results.to_csv(os.path.join(config.predictions_hybrid_arimax_lstm, f'arimax_lstm_{variant_co2}_test.csv'))

## Charts

In [13]:
charts = ModelCharts(train_results, test_results)

#Line plot
charts.generate_line_plot(variant = variant_co2, model_output_file=config.predictions_hybrid_arimax_lstm)
charts.generate_line_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_hybrid_arimax_lstm, dataset_type='train')
charts.generate_line_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_hybrid_arimax_lstm, dataset_type='test')

#Scatter plot
charts.generate_scatter_plot(variant = variant_co2, model_output_file=config.predictions_hybrid_arimax_lstm)
charts.generate_scatter_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_hybrid_arimax_lstm, dataset_type='train')
charts.generate_scatter_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_hybrid_arimax_lstm, dataset_type='test')

## Metrics

In [14]:
evaluator = PredictionEvaluator()
evaluator.evaluate(train_results, test_results, actual_col='co2_actual', predicted_col='co2_predicted', variant = variant_co2, model_output_file=config.metrics_hybrid)